## Example Use of David's Finance Module, dfinance
#### See it in action!
This notebook shows an example that implements my custom finance package to backtest an (ineffective) algorithmic trading strategy.

In [8]:
import pandas as pd
import yfinance as yf
import numpy as np
import pickle

import dfinance as dfin
from dfinance.trading_strategies import SMA_Crossover
from dfinance.trading_strategies import PaperBracketOrder
from dfinance.trading_strategies import BTBracketOrder

import talib
from talib.abstract import *

from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Backtest a strategy that buys based on model predictions

In [9]:
my_port = dfin.Portfolio()
my_back = dfin.Backtest()

In [10]:
# Get cash value - can also set this
my_port.cashvalue = 1000

In [11]:
# load data from Yahoo!
days_of_data = 110

data = yf.download(tickers='HOG', period=f'{days_of_data}d', progress=False)
data = data.head(days_of_data - 10)

In [12]:
my_back.process_historical_data(df=data,
                                my_port=my_port,
                                my_strategy=BTBracketOrder.strategy,
                                ticker='HOG',
                                share_cnt=25,
                                profit_pct=1.05,
                                stop_pct=0.96,
                                limit_pct=0.95,
                                pkl_filename='lgbm_model.pkl')

Buying 25 shares of HOG for 927.0000457763672
Selling 25 shares of HOG for 934.7499847412109
Buying 25 shares of HOG for 914.0000343322754
Selling 25 shares of HOG for 905.5000305175781
Buying 25 shares of HOG for 916.7499542236328
Selling 25 shares of HOG for 912.7499580383301
Buying 25 shares of HOG for 882.7500343322754
Selling 25 shares of HOG for 907.4999809265137
Buying 25 shares of HOG for 907.4999809265137
Selling 25 shares of HOG for 923.2500076293945
Buying 25 shares of HOG for 914.5000457763672
Selling 25 shares of HOG for 901.2499809265137
Buying 25 shares of HOG for 901.0000228881836
Selling 25 shares of HOG for 932.2500228881836
Buying 25 shares of HOG for 915.7500267028809
Selling 25 shares of HOG for 912.7499580383301
Buying 25 shares of HOG for 905.0000190734863
Selling 25 shares of HOG for 904.5000076293945
Buying 25 shares of HOG for 917.2499656677246
Selling 25 shares of HOG for 916.2500381469727
Buying 25 shares of HOG for 885.9999656677246
Selling 25 shares of HOG

In [13]:
my_port.cashvalue

977.9998779296875

In [14]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,HOG,25,37.080002,927.000046,buy,2021-09-13
1,HOG,-25,37.389999,-934.749985,sell,2021-09-14
2,HOG,25,36.560001,914.000034,buy,2021-09-30
3,HOG,-25,36.220001,-905.500031,sell,2021-10-01
4,HOG,25,36.669998,916.749954,buy,2021-10-04
5,HOG,-25,36.509998,-912.749958,sell,2021-10-05
6,HOG,25,35.310001,882.750034,buy,2021-10-06
7,HOG,-25,36.299999,-907.499981,sell,2021-10-07
8,HOG,25,36.299999,907.499981,buy,2021-10-08
9,HOG,-25,36.930000,-923.250008,sell,2021-10-11


---

### Paper trading with Alpaca
Only works when the market is open.

In [15]:
my_alpaca_port = dfin.AlpacaPortfolio()
my_back = dfin.Backtest()

In [16]:
prefill_rows = yf.download(tickers='HOG', period='100d', progress=False)

In [17]:
my_back.process_live_data(prefill_rows=prefill_rows,
                          my_alpaca_port=my_alpaca_port,
                          my_strategy=PaperBracketOrder.strategy,
                          ticker='HOG',
                          share_cnt=3,
                          profit_pct=1.05,
                          stop_pct=0.95,
                          limit_pct=0.94,
                          pkl_filename='lgbm_model.pkl')

Market is closed. No action will be taken.


In [ ]:
# must call refresh() before getting status or positions
# refresh() reloads the value of all attributes with current values
my_alpaca_port.refresh()

my_alpaca_port.get_account_status()

,Cash,Equity,Last Equity,Equity Change,Equity Change %
0,99999.11,99999.11,99999.11,0.0,0.0


In [68]:
my_alpaca_port.current_positions()

,Ticker,Shares Held,Cost Basis,Current Price,Unrealized Gain/Loss,Unrealized Gain/Loss %


---

### Backtest SMA Crossover strategy

In [107]:
# Initialize backtest object
# Initialize portfolio object
my_back = dfin.Backtest()
my_port = dfin.Portfolio()

In [108]:
# Get cash value - can also set this
my_port.cashvalue

100000.0

In [109]:
# load data from Yahoo!
data = yf.download(tickers='AAPL', period='90d', progress=False)
data.to_csv('aapl_daily_data.csv')
historical_df = pd.read_csv('aapl_daily_data.csv', index_col='Date')

In [110]:
# Process the data from the historical dataframe
my_back.process_historical_data(historical_df,
                                my_port,
                                SMA_Crossover.strategy,
                                sma_short=10,
                                sma_long=15,
                                share_num=10,
                                ticker='AAPL')

2021-10-05 | Short: 142.05 | Long: 142.92 | Short < Long
2021-10-06 | Short: 141.51 | Long: 142.39 | Short < Long
2021-10-07 | Short: 141.22 | Long: 142.09 | Short < Long
2021-10-08 | Short: 140.92 | Long: 141.88 | Short < Long
2021-10-11 | Short: 140.72 | Long: 141.91 | Short < Long
2021-10-12 | Short: 140.65 | Long: 141.80 | Short < Long
2021-10-13 | Short: 140.37 | Long: 141.50 | Short < Long
2021-10-14 | Short: 140.39 | Long: 141.22 | Short < Long
2021-10-15 | Short: 140.83 | Long: 141.09 | Short < Long
2021-10-18 | Short: 141.32 | Long: 141.04 | Short > Long

Buying 10 shares of AAPL for 1431.6000366210938

2021-10-19 | Short: 142.04 | Long: 141.37 | Short > Long
2021-10-20 | Short: 143.02 | Long: 141.77 | Short > Long
2021-10-21 | Short: 143.53 | Long: 142.21 | Short > Long
2021-10-22 | Short: 144.14 | Long: 142.85 | Short > Long
2021-10-25 | Short: 144.72 | Long: 143.47 | Short > Long
2021-10-26 | Short: 145.52 | Long: 144.11 | Short > Long
2021-10-27 | Short: 146.45 | Long: 144

In [111]:
my_port.cashvalue

100329.3000793457

In [112]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,AAPL,10,143.160004,1431.600037,buy,2021-10-18
1,AAPL,-10,178.529999,-1785.299988,sell,2021-12-28
2,AAPL,10,177.259995,1772.599945,buy,2021-12-31
3,AAPL,-10,174.820007,-1748.200073,sell,2022-01-12


### Demonstrate other functions of the Porfolio class

In [113]:
my_port.current_positions()

,Ticker,Shares Held,Current Price,Current Value of Shares


In [114]:
# Buy one share of GME for $100 - what a great price!
my_port.buy_stock('GME', 3, 100, '2021-01-01 00:00:00')

In [115]:
my_port.save_transactions('my_port.csv')

In [116]:
my_port.load_transactions('my_port.csv')
my_port.current_positions()

,Ticker,Shares Held,Current Price,Current Value of Shares
1,GME,3,102.235001,306.705002
2,Totals,3,N/A,306.705002
